Compute water occurrence for each dataframe points based on 'users/gena/AquaMonitor2020/bathymetry-jamuna', append the column to the dataframe, and save them to 
df.to_pickle('../data/out/df.pkl')
df2.to_pickle('../data/out/df2.pkl')
df3.to_pickle('../data/out/df3.pkl') 

In [1]:
%matplotlib widget

In [2]:
%matplotlib widget
# %matplotlib inline
# %matplotlib qt 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import shape, GeometryCollection
from pyproj import CRS
from tqdm.notebook import tqdm
from tqdm import trange
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
    
#import geemap.eefolium as emap
import geemap as emap
create_map = lambda: emap.Map()

#ndvi files from earthengine; does not exist anyore
#no need to query images one by one

## Query water occurrence values for all ICESat-2 points

In [3]:
water_occurance_jamuna = ee.Image('users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019')

In [4]:
water_occurance_jamuna = ee.Image('users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019')
#actually has image collection of all images where we generated intertidal water occurance; intended for coastline
#replace this with one we generated for Jamuna
#Say for jamuna we have for one year; every year different; match icesat with water occurance from same year
#water occurance shared here available for larger areas

#bathymetry_from_space.. website
#https://gena.users.earthengine.app/view/bathymetry-fusion

In [5]:
water_occurance_jamuna

In [6]:
water_occurance_jamuna.geometry().bounds().getInfo()
#coordinates also for first script
#download data for this extent in the first script
#learn to work with this data
#pull water occurance data for every icesat point; analyse or same it maybe in same parquet file

#filter icesat points by date

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[89.57958670206244, 23.721136875400937],
   [89.99453684543461, 23.721136875400937],
   [89.99453684543461, 24.618409448937893],
   [89.57958670206244, 24.618409448937893],
   [89.57958670206244, 23.721136875400937]]]}

In [7]:
m = create_map()
m.addLayer(water_occurance_jamuna.select('ndwi'), { 'min': -0.10, 'max': 0.15, 'palette': ['lightblue', 'black'] }, 'water occurrence')
#m.addLayer(extent, { 'color': 'white' }, 'rect')
#m.addLayer(pointsRGB, {}, 'sampled points')
#m.setCenter(df.lon.mean(), df.lat.mean(), 11)
m.setCenter((89.99453684543461+89.57958670206244)/2,(23.721136875400937+24.618409448937893)/2,10)
m.addLayerControl()
m

Map(center=[24.169773162169413, 89.78706177374852], controls=(WidgetControl(options=['position'], widget=HBox(…

In [8]:
df = pd.read_parquet('../data/out/icesat-2-BGD.parquet')
df

,track_id,date,beam,series,lon,lat,h
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720
...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422


In [9]:
image = ee.Image('users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019').select('ndwi')
print(image.getInfo())

extent = [89.57958670206244, 23.721136875400937, 89.99453684543461, 24.618409448937893]
rect = ee.Geometry.Rectangle(extent)
print(rect)



{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4618, 10947], 'crs': 'EPSG:3857', 'crs_transform': [10, 0, 9971960, 0, -10, 2828940]}], 'version': 1591280732867259, 'id': 'users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019', 'properties': {'system:time_start': 1533081600000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[89.57958979474114, 24.61833286303767], [89.57958670206244, 24.618318084856465], [89.57958742129006, 23.721226920090466], [89.57962741032551, 23.72118544409774], [89.57966344357631, 23.72113687540096], [89.68337378067785, 23.721136926357552], [89.78706181738652, 23.721136899916807], [89.8907498200214, 23.721136949501748], [89.99443786455846, 23.72113689504871], [89.99448319228122, 23.72117348770924], [89.99453620501279, 23.721206448758878], [89.99453684543461, 24.618310598395617], [89.99449664115566, 24.61835200694045], [89.99446248527484, 24.618397765100205], [89.99

In [10]:
image = image.clip(rect)
print(image)

ee.Image({
  "type": "Invocation",
  "arguments": {
    "input": {
      "type": "Invocation",
      "arguments": {
        "input": {
          "type": "Invocation",
          "arguments": {
            "id": "users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019"
          },
          "functionName": "Image.load"
        },
        "bandSelectors": [
          "ndwi"
        ]
      },
      "functionName": "Image.select"
    },
    "geometry": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            89.57958670206244,
            24.618409448937893
          ],
          [
            89.57958670206244,
            23.721136875400937
          ],
          [
            89.99453684543461,
            23.721136875400937
          ],
          [
            89.99453684543461,
            24.618409448937893
          ]
        ]
      ],
      "evenOdd": true
    }
  },
  "functionName": "Image.clip"
})


In [11]:
m.setCenter((89.99453684543461+89.57958670206244)/2,(23.721136875400937+24.618409448937893)/2,10)
m.addLayerControl()
m
# Map_as = emap.Map(center=[40,-100], zoom=4)
# Map_as

Map(bottom=113201.0, center=[24.169773162169413, 89.78706177374852], controls=(WidgetControl(options=['positio…

In [12]:
# # Add Earth Engine dataset
# #wqs_script
# fromFT = ee.FeatureCollection("users/gena/AquaMonitor2020/bathymetry-jamuna")
# # This function computes the feature's geometry area and adds it as a property.
# def addArea(feature):
#   return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})


# # Map the area getting function over the FeatureCollection.
# areaAdded = fromFT.map(addArea)
# # Print the first feature from the collection with the added property.

# first = areaAdded.first()
# print('First feature: ', first.getInfo())
# print("areaHa: ", first.get("areaHa").getInfo())

In [13]:
# Map_as.addLayerControl() # This line is not needed for ipyleaflet-based Map.
# Map_as



In [14]:
# # DK
# # df = pd.read_parquet('../data/out/icesat-2-DK.parquet')
# # image = ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4521-v9') 

# # NL
# #df = pd.read_parquet('../data/out/icesat-2-NL.parquet')
# #extent = [5.8062744140625, 53.36202615413913, 6.1791229248046875, 53.5092846887053]
# #rect = ee.Geometry.Rectangle(extent)
# #image = ee.ImageCollection([ 
# #    ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4520-v9').resample('bicubic'),
# #    ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4448-v9').resample('bicubic')
# #]).mosaic().clip(rect)

# # FR
# #df = pd.read_parquet('../data/out/icesat-2-FR.parquet')


# #Bangladesh
# df = pd.read_parquet('../data/out/icesat-2-BGD.parquet')
# #image = ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4300-v9')
# image = ee.Image('users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019').select('ndwi')
# #df = pd.read_parquet('../data/out/icesat-2-NL.parquet')
# #extent = [5.8062744140625, 53.36202615413913, 6.1791229248046875, 53.5092846887053]
# extent = [89.57958670206244, 23.721136875400937, 89.99453684543461, 24.618409448937893] #Extent same as users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019

# rect = ee.Geometry.Rectangle(extent)

# image = image.clip(rect)
# #image = ee.ImageCollection([ 
# #    ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4520-v9').resample('bicubic'),
# #    ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4448-v9').resample('bicubic')
# #]).mosaic().clip(rect)

In [15]:
#print(image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=image.geometry(),scale=90))

In [16]:
print(image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=image.geometry(),scale=90).getInfo())

{'ndwi_max': 0.15000204193247493, 'ndwi_min': -0.05000136165685305}


In [17]:
#print(type(image))
#print(image.propertyNames())

In [18]:
#This line is initially uncommented

print(image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=image.geometry(), scale=90).getInfo())

{'ndwi_max': 0.15000204193247493, 'ndwi_min': -0.05000136165685305}


In [19]:
df.columns

Index(['track_id', 'date', 'beam', 'series', 'lon', 'lat', 'h'], dtype='object')

In [20]:
df.series.unique()

array(['Noise', 'Buffer', 'Low', 'Medium', 'High'], dtype=object)

In [21]:
BATCH_SIZE = 50000 # fails after ~90k

In [22]:
len(df) // BATCH_SIZE

83

In [23]:
# df2 = df_high.iloc[:BATCH_SIZE,:]

In [24]:
df2 = df.sample(n=BATCH_SIZE)

In [25]:
#IceSAT2 Data
df

,track_id,date,beam,series,lon,lat,h
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720
...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422


In [26]:
df2

,track_id,date,beam,series,lon,lat,h
2935544,1239,2019-03-19,gt1r,High,89.864862,23.950709,-47.752160
2616108,797,2019-11-18,gt1r,Medium,89.752211,24.338253,-42.854190
2909085,1239,2019-03-19,gt1r,Medium,89.855900,23.867250,-45.273254
1646421,622,2020-05-06,gt1r,Noise,89.677682,24.431684,-108.156265
1759607,797,2018-11-19,gt2r,High,89.622139,24.552785,-44.423004
...,...,...,...,...,...,...,...
3847341,1239,2019-12-17,gt1l,Noise,89.980850,24.040977,-24.295998
4085473,1239,2020-03-16,gt1l,High,89.827906,24.431993,-42.282050
1239271,622,2020-02-05,gt2r,Noise,89.888936,23.968182,146.139160
4012789,1239,2020-03-16,gt2l,High,89.763225,24.123039,-50.180340


In [27]:
len(df)

4150043

In [28]:
df2 = df[(df.series != 'Noise')].sample(n=BATCH_SIZE)

In [29]:
df2

,track_id,date,beam,series,lon,lat,h
204288,180,2019-04-10,gt3r,Low,89.592016,23.999611,-50.773170
2794859,1239,2019-03-19,gt2r,High,89.903481,24.603572,-41.336563
539077,622,2019-02-07,gt2r,High,89.805811,24.150799,-50.132454
1409400,622,2020-02-05,gt1l,Buffer,89.841831,24.105544,-35.754270
4079267,1239,2020-03-16,gt1r,High,89.755876,23.766953,-52.853855
...,...,...,...,...,...,...,...
2808827,1239,2019-03-19,gt2r,High,89.887589,24.455773,-42.138386
1976954,797,2019-02-18,gt2r,High,89.650822,24.564696,-41.503963
2776171,1239,2019-03-19,gt3l,High,89.816097,24.076185,-46.041965
1276106,622,2020-02-05,gt2r,High,89.857869,24.259231,-43.395878


In [30]:
coords = list(zip(df2.lon, df2.lat))
coords

[(89.592016, 23.999611),
 (89.903481, 24.603572),
 (89.805811, 24.150799),
 (89.841831, 24.105544),
 (89.755876, 23.766953),
 (89.772402, 24.218172),
 (89.656773, 24.601873),
 (89.652366, 24.284617),
 (89.78849, 24.064556),
 (89.762987, 23.826109),
 (89.81294, 24.291435),
 (89.742172, 23.808238),
 (89.580484, 24.099476),
 (89.846036, 24.361526),
 (89.607585, 24.154265),
 (89.595856, 23.955373),
 (89.587609, 23.753974),
 (89.769211, 24.180115),
 (89.800641, 23.931805),
 (89.597947, 24.112767),
 (89.797366, 23.909261),
 (89.580939, 23.816271),
 (89.92497, 24.11722),
 (89.63247, 24.246463),
 (89.606766, 24.178967),
 (89.61508, 24.224117),
 (89.580273, 24.131411),
 (89.639799, 24.176612),
 (89.88985, 24.468932),
 (89.761124, 24.567001),
 (89.886831, 24.448718),
 (89.807395, 24.003049),
 (89.979072, 24.327994),
 (89.62407, 24.27569),
 (89.861825, 24.510912),
 (89.861452, 23.91801),
 (89.8806, 23.743163),
 (89.635476, 24.127603),
 (89.587472, 23.755232),
 (89.759322, 24.39669),
 (89.763405, 

In [31]:
len(df2)

50000

In [32]:
len(coords)

50000

In [33]:
#Debug Lines
g = ee.Geometry.MultiPoint(coords) # visualize using multi-point instead of feature collection - faster
pointsRGB = ee.FeatureCollection(g).style(pointSize=2, color='00000000', fillColor='ffeda0')
pointsRGB = pointsRGB.select(0).mask().focal_max(3).selfMask().visualize(palette=['000000'], opacity=0.5).blend(pointsRGB)

extent = ee.Geometry.Rectangle(coords=[df.lon.min(), df.lat.min(), df.lon.max(), df.lat.max()]) 
extent = ee.Geometry.LineString(extent.coordinates().get(0))

In [34]:
imageRGB = image.updateMask(image.gt(-0.09)) 

In [35]:
print(imageRGB)

ee.Image({
  "type": "Invocation",
  "arguments": {
    "image": {
      "type": "Invocation",
      "arguments": {
        "input": {
          "type": "Invocation",
          "arguments": {
            "input": {
              "type": "Invocation",
              "arguments": {
                "id": "users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019"
              },
              "functionName": "Image.load"
            },
            "bandSelectors": [
              "ndwi"
            ]
          },
          "functionName": "Image.select"
        },
        "geometry": {
          "type": "Polygon",
          "coordinates": [
            [
              [
                89.57958670206244,
                24.618409448937893
              ],
              [
                89.57958670206244,
                23.721136875400937
              ],
              [
                89.99453684543461,
                23.721136875400937
              ],
              [

In [36]:
print(imageRGB.bandNames().getInfo())

['ndwi']


In [37]:
m = create_map()
m.addLayer(imageRGB.select('ndwi'), { 'min': -0.10, 'max': 0.15, 'palette': ['lightblue', 'black'] }, 'water occurrence')
m.addLayer(extent, { 'color': 'white' }, 'rect')
m.addLayer(pointsRGB, {}, 'sampled points')
m.setCenter(df.lon.mean(), df.lat.mean(), 11)
m.addLayerControl()
m
#End of Debug Lines

Map(center=[24.16908007069371, 89.74071838375434], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [38]:
g = ee.Geometry.MultiPoint(coords) # visualize using multi-point instead of feature collection - faster
pointsRGB = ee.FeatureCollection(g).style(pointSize=2, color='00000000', fillColor='ffeda0')
pointsRGB = pointsRGB.select(0).mask().focal_max(3).selfMask().visualize(palette=['000000'], opacity=0.5).blend(pointsRGB)

extent = ee.Geometry.Rectangle(coords=[df.lon.min(), df.lat.min(), df.lon.max(), df.lat.max()]) 
extent = ee.Geometry.LineString(extent.coordinates().get(0))

imageRGB = image.updateMask(image.gt(-0.09)) 

m = create_map()
m.addLayer(imageRGB.select('ndwi'), { 'min': -0.10, 'max': 0.15, 'palette': ['lightblue', 'black'] }, 'water occurrence')
m.addLayer(extent, { 'color': 'white' }, 'rect')
m.addLayer(pointsRGB, {}, 'sampled points')
m.setCenter(df.lon.mean(), df.lat.mean(), 11)
m.addLayerControl()
m

Map(center=[24.16908007069371, 89.74071838375434], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [39]:
print(image)

ee.Image({
  "type": "Invocation",
  "arguments": {
    "input": {
      "type": "Invocation",
      "arguments": {
        "input": {
          "type": "Invocation",
          "arguments": {
            "id": "users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019"
          },
          "functionName": "Image.load"
        },
        "bandSelectors": [
          "ndwi"
        ]
      },
      "functionName": "Image.select"
    },
    "geometry": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            89.57958670206244,
            24.618409448937893
          ],
          [
            89.57958670206244,
            23.721136875400937
          ],
          [
            89.99453684543461,
            23.721136875400937
          ],
          [
            89.99453684543461,
            24.618409448937893
          ]
        ]
      ],
      "evenOdd": true
    }
  },
  "functionName": "Image.clip"
})


In [40]:
water_occurance_jamuna2 = ee.ImageCollection('users/gena/AquaMonitor2020/bathymetry-jamuna')

In [41]:
results = []
print(water_occurance_jamuna2)

ee.ImageCollection({
  "type": "Invocation",
  "arguments": {
    "id": "users/gena/AquaMonitor2020/bathymetry-jamuna"
  },
  "functionName": "ImageCollection.load"
})


In [42]:
results = []
i = 0
coords = list(zip(df.lon.values[i:(i+BATCH_SIZE)], df.lat.values[i:(i+BATCH_SIZE)]))
features = ee.List(coords).map(lambda o: ee.Feature(ee.Geometry.Point(coords=o)))
features = ee.FeatureCollection(features)
values = image.unmask(-999, False).reduceRegions(collection=features, reducer=ee.Reducer.first().setOutputs(['water_occurrence']), scale=10, tileScale=1)
results = results + values.aggregate_array('water_occurrence').getInfo()

In [43]:
print(results)

[-0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.05, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0

In [44]:
print(coords)

[(89.622678, 23.727912), (89.62268, 23.727912), (89.622685, 23.727913), (89.622679, 23.727918), (89.622678, 23.727918), (89.622684, 23.727919), (89.622684, 23.727919), (89.622683, 23.727919), (89.622678, 23.727925), (89.622677, 23.727931), (89.622676, 23.727937), (89.622682, 23.727938), (89.62268, 23.727951), (89.62268, 23.727951), (89.622678, 23.727957), (89.622671, 23.727969), (89.622672, 23.727969), (89.622673, 23.727982), (89.622676, 23.727983), (89.622674, 23.727995), (89.622668, 23.728007), (89.622666, 23.72802), (89.622668, 23.72802), (89.622663, 23.728046), (89.622665, 23.728046), (89.622664, 23.728052), (89.62267, 23.728053), (89.622666, 23.728065), (89.622661, 23.728078), (89.622663, 23.728091), (89.622666, 23.728091), (89.622664, 23.728098), (89.622662, 23.728104), (89.622657, 23.72811), (89.622659, 23.72811), (89.622659, 23.72811), (89.622661, 23.72811), (89.622661, 23.72811), (89.622663, 23.72811), (89.622658, 23.728116), (89.622661, 23.728117), (89.622658, 23.728123), (89

In [45]:
print(values)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
print(values.aggregate_array('water_occurrence').getInfo())

[-0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.04984868511268281, -0.05, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000001, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0.05000000000000005, -0

In [47]:
print(values.aggregate_array('water_occurrence'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
results = []

for i in tqdm(range(0, len(df), BATCH_SIZE)):
    coords = list(zip(df.lon.values[i:(i+BATCH_SIZE)], df.lat.values[i:(i+BATCH_SIZE)]))
    
    features = ee.List(coords).map(lambda o: ee.Feature(ee.Geometry.Point(coords=o)))
    #print(features)
    features = ee.FeatureCollection(features)
    
    values = image.unmask(-999, False).reduceRegions(collection=features, reducer=ee.Reducer.first().setOutputs(['water_occurrence']), scale=10, tileScale=1)
    results = results + values.aggregate_array('water_occurrence').getInfo()

In [49]:
print(features)

ee.FeatureCollection({
  "type": "Invocation",
  "arguments": {
    "features": {
      "type": "Invocation",
      "arguments": {
        "list": [
          [
            89.751951,
            23.722707
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.751949,
            23.722694
          ],
          [
            89.751949,
            23.722694
          ],
          [
            89.751948,
            23.722687
          ],
          [
            89.751948,
            23.722681
          ],
          [
            89.751947,
            23.722675
          ],
          [
            89.751947,
            23.722675
          ],
          [
            89.751946,
            23.722668
          ],
          [
            89.751946,
            23.722662
          ],
          [
        

In [50]:
print(results)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
print(features)

ee.FeatureCollection({
  "type": "Invocation",
  "arguments": {
    "features": {
      "type": "Invocation",
      "arguments": {
        "list": [
          [
            89.751951,
            23.722707
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.75195,
            23.7227
          ],
          [
            89.751949,
            23.722694
          ],
          [
            89.751949,
            23.722694
          ],
          [
            89.751948,
            23.722687
          ],
          [
            89.751948,
            23.722681
          ],
          [
            89.751947,
            23.722675
          ],
          [
            89.751947,
            23.722675
          ],
          [
            89.751946,
            23.722668
          ],
          [
            89.751946,
            23.722662
          ],
          [
        

In [52]:
print(features.getInfo)

<bound method Collection.getInfo of <ee.featurecollection.FeatureCollection object at 0x7fa3a886a1d0>>


In [53]:
print(values)

ee.FeatureCollection({
  "type": "Invocation",
  "arguments": {
    "image": {
      "type": "Invocation",
      "arguments": {
        "input": {
          "type": "Invocation",
          "arguments": {
            "input": {
              "type": "Invocation",
              "arguments": {
                "input": {
                  "type": "Invocation",
                  "arguments": {
                    "id": "users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019"
                  },
                  "functionName": "Image.load"
                },
                "bandSelectors": [
                  "ndwi"
                ]
              },
              "functionName": "Image.select"
            },
            "geometry": {
              "type": "Polygon",
              "coordinates": [
                [
                  [
                    89.57958670206244,
                    24.618409448937893
                  ],
                  [
                 

In [54]:
print(values.getInfo)

<bound method Collection.getInfo of <ee.featurecollection.FeatureCollection object at 0x7fa3a886a8d0>>


In [55]:
len(results)

4150043

In [56]:
df['water_occurrence'] = results

In [57]:
df

,track_id,date,beam,series,lon,lat,h,water_occurrence
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254,-0.050000
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510,-0.050000
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760,-0.050000
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028,-0.050000
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720,-0.050000
...,...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958,-0.049445
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526,-0.049445
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990,-0.049841
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422,-0.049955


In [58]:
df[df.water_occurrence == -999] = np.nan

In [59]:
df

,track_id,date,beam,series,lon,lat,h,water_occurrence
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254,-0.050000
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510,-0.050000
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760,-0.050000
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028,-0.050000
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720,-0.050000
...,...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958,-0.049445
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526,-0.049445
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990,-0.049841
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422,-0.049955


In [60]:
fig, ax = plt.subplots()
df.water_occurrence.hist(ax=ax, bins=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [61]:
# plt.close('all')
fig, ax = plt.subplots(figsize=(10, 3))
hmin = 45
hmax = 49
# df3 = df[(df.h > hmin) & (df.h < hmax)]
df3 = df

df4 = df3[df3.series == 'High']
ax.plot(df4.water_occurrence, df4.h, 'k.', markersize=0.5, alpha=0.2)

df4 = df3[df3.series == 'Medium']
ax.plot(df4.water_occurrence, df4.h, 'r.', markersize=1)

df4 = df3[df3.series == 'Buffer']
ax.plot(df4.water_occurrence, df4.h, 'b.', markersize=1)

ax.set_xlabel('water occurrence')
ax.set_ylabel('elevation (ICESat-2)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
df2 = df
df2

,track_id,date,beam,series,lon,lat,h,water_occurrence
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254,-0.050000
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510,-0.050000
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760,-0.050000
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028,-0.050000
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720,-0.050000
...,...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958,-0.049445
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526,-0.049445
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990,-0.049841
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422,-0.049955


In [63]:
df2.beam.unique()

array(['gt3r', nan, 'gt3l', 'gt2r', 'gt2l', 'gt1r', 'gt1l'], dtype=object)

In [64]:
tracks = df.track_id.unique()
tracks

array(['180', nan, '622', '797', '1239'], dtype=object)

In [65]:
for t in tracks:
    print(t, df[df.track_id == t].date.unique())

180 ['2019-01-09' '2019-04-10' '2020-01-07']
nan []
622 ['2019-02-07' '2019-05-08' '2019-08-07' '2019-11-06' '2020-02-05'
 '2020-05-06']
797 ['2018-11-19' '2019-02-18' '2019-05-20' '2019-11-18' '2020-02-16']
1239 ['2018-12-18' '2019-03-19' '2019-06-18' '2019-09-17' '2019-12-17'
 '2020-03-16']


In [66]:
df.date.unique()

array(['2019-01-09', nan, '2019-04-10', '2020-01-07', '2019-02-07',
       '2019-05-08', '2019-08-07', '2019-11-06', '2020-02-05',
       '2020-05-06', '2018-11-19', '2019-02-18', '2019-05-20',
       '2019-11-18', '2020-02-16', '2018-12-18', '2019-03-19',
       '2019-06-18', '2019-09-17', '2019-12-17', '2020-03-16'],
      dtype=object)

In [67]:
tqdm.pandas()

/home/shakya/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [68]:
# hmin = 45
# hmax = 49
# df3 = df[(df.h > hmin) & (df.h < hmax)].copy()
df3 = df

df3['filter'] = df3.progress_apply(lambda row: f"{row['track_id']}, {row['beam']}, {row['series']}", axis=1).values


In [69]:
df3

,track_id,date,beam,series,lon,lat,h,water_occurrence,filter
0,180,2019-01-09,gt3r,Noise,89.622678,23.727912,3.698254,-0.050000,"180, gt3r, Noise"
1,180,2019-01-09,gt3r,Noise,89.622680,23.727912,-30.436510,-0.050000,"180, gt3r, Noise"
2,180,2019-01-09,gt3r,Noise,89.622685,23.727913,-103.526760,-0.050000,"180, gt3r, Noise"
3,180,2019-01-09,gt3r,Noise,89.622679,23.727918,-19.456028,-0.050000,"180, gt3r, Noise"
4,180,2019-01-09,gt3r,Noise,89.622678,23.727918,-9.291720,-0.050000,"180, gt3r, Noise"
...,...,...,...,...,...,...,...,...,...
4150038,1239,2020-03-16,gt1l,High,89.751855,23.721827,-47.611958,-0.049445,"1239, gt1l, High"
4150039,1239,2020-03-16,gt1l,High,89.751852,23.721802,-47.792526,-0.049445,"1239, gt1l, High"
4150040,1239,2020-03-16,gt1l,High,89.751844,23.721725,-47.442990,-0.049841,"1239, gt1l, High"
4150041,1239,2020-03-16,gt1l,High,89.751837,23.721668,-47.488422,-0.049955,"1239, gt1l, High"


In [70]:
filters = df3.sort_values(['track_id', 'beam', 'series'])['filter'].drop_duplicates().values

In [71]:
filters = [f for f in filters if 'High' in f]

In [72]:
filters

['1239, gt1l, High',
 '1239, gt1r, High',
 '1239, gt2l, High',
 '1239, gt2r, High',
 '1239, gt3l, High',
 '1239, gt3r, High',
 '180, gt1l, High',
 '180, gt1r, High',
 '180, gt2l, High',
 '180, gt2r, High',
 '180, gt3l, High',
 '180, gt3r, High',
 '622, gt1l, High',
 '622, gt1r, High',
 '622, gt2l, High',
 '622, gt2r, High',
 '622, gt3l, High',
 '622, gt3r, High',
 '797, gt1l, High',
 '797, gt1r, High',
 '797, gt2l, High',
 '797, gt2r, High',
 '797, gt3l, High',
 '797, gt3r, High']

In [73]:
df.columns

Index(['track_id', 'date', 'beam', 'series', 'lon', 'lat', 'h',
       'water_occurrence', 'filter'],
      dtype='object')

In [74]:
hmin = 38
hmax = 44

df3 = df[(df.series != 'Noise') & (df.series != 'Medium')]
df3 = df3[(df3.h > hmin) & (df3.h < hmax)]



@interact(f=filters)
def show(f=filters[0]):
    plt.close('all')
    fig, ax = plt.subplots(figsize=(13, 5))

    # draw all
    ax.plot(df3.water_occurrence, df3.h, 'k.', markersize=0.5, alpha=0.1)

    # draw only specific track/beam/series
    df4 = df3[df3['filter'] == f]
    ax.plot(df4.water_occurrence, df4.h, 'r.', markersize=1, alpha=0.1)

    ax.set_xlabel('water occurrence')
    ax.set_ylabel('elevation (ICESat-2)')
    plt.tight_layout()

interactive(children=(Dropdown(description='f', options=('1239, gt1l, High', '1239, gt1r, High', '1239, gt2l, …

In [75]:
df.to_pickle('../data/out/df.pkl')
df2.to_pickle('../data/out/df2.pkl')
df3.to_pickle('../data/out/df3.pkl')